In [ ]:
# ============================================================
# 📌 1. INSTALL DEPENDENCIES (COLAB)
# ============================================================

!pip install transformers datasets scikit-learn pandas numpy accelerate

import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder
import torch
import json
from google.colab import files

print("✔ Dependencies installed.")


# ============================================================
# 📌 2. UPLOAD YOUR CSV FILE
# ============================================================

print("Combined Data.csv")
uploaded = files.upload()

# Detect uploaded CSV filename automatically
csv_filename = list(uploaded.keys())[0]
print("✔ File uploaded:", csv_filename)


# ============================================================
# 📌 3. LOAD DATASET
# ============================================================

df = pd.read_csv(csv_filename)

# Expected columns:
#   statement → input text
#   status    → output label
df = df[['statement', 'status']].dropna()

print("✔ Dataset loaded with shape:", df.shape)
df.head()


# ============================================================
# 📌 4. ENCODE LABELS
# ============================================================

le = LabelEncoder()
df["label"] = le.fit_transform(df["status"])

print("✔ Label Mapping:")
label_mapping = {cls: int(num) for cls, num in zip(le.classes_, le.transform(le.classes_))}
label_mapping


# Save label mapping file
with open("text_model_label_mapping.json", "w") as f:
    json.dump(label_mapping, f, indent=4)

print("✔ Label mapping saved → text_model_label_mapping.json")


# ============================================================
# 📌 5. PREPARE HUGGINGFACE DATASET
# ============================================================

dataset = Dataset.from_pandas(df[['statement', 'label']])

MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    return tokenizer(
        batch["statement"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.remove_columns(["statement"])
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch")

dataset = dataset.train_test_split(test_size=0.1)
train_ds = dataset["train"]
test_ds = dataset["test"]

print("✔ Dataset tokenized and split.")


# ============================================================
# 📌 6. LOAD MODEL
# ============================================================

num_labels = len(le.classes_)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
)

print("✔ Model loaded with", num_labels, "output classes.")


# ============================================================
# 📌 7. TRAINING SETUP (Colab compatible)
# ============================================================

training_args = TrainingArguments(
    output_dir="text_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    logging_steps=50,
    learning_rate=3e-5,
    do_eval=True,
    save_steps=5000,
    eval_steps=5000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

print("✔ Trainer initialized.")


# ============================================================
# 📌 8. TRAIN MODEL
# ============================================================

trainer.train()

print("✔ Training complete!")


# ============================================================
# 📌 9. SAVE MODEL + TOKENIZER
# ============================================================

model.save_pretrained("text_model")
tokenizer.save_pretrained("text_model")

print("✔ Model saved to text_model/")


# ============================================================
# 📌 10. ZIP AND DOWNLOAD MODEL
# ============================================================

!zip -r text_model.zip text_model
files.download("text_model.zip")
files.download("text_model_label_mapping.json")

print("🎉 All files ready for download!")


✔ Dependencies installed.
Combined Data.csv


Saving Combined Data.csv to Combined Data (5).csv
✔ File uploaded: Combined Data (5).csv
✔ Dataset loaded with shape: (52681, 2)
✔ Label Mapping:
✔ Label mapping saved → text_model_label_mapping.json


Map:   0%|          | 0/52681 [00:00<?, ? examples/s]

✔ Dataset tokenized and split.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 All files ready for download!
